#### Viernes 16 de mayo de 2025

## Reglas de Asociación. Algoritmo Apriori. Análisis de Ventas en una Librería

In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

#### Contexto: Eres el gerente de Libros & Más, una librería que quiere optimizar sus recomendaciones y disposición de productos. Tienes datos de 10 transacciones recientes:

In [3]:
# Transacciones

transacciones = [
    ['Novela', 'Café', 'Marcadores'],
    ['Libro de Ciencia', 'Cuaderno', 'Lápiz'],
    ['Novela', 'Café', 'Lápiz'],
    ['Libro de Historia', 'Marcadores', 'Post-it'],
    ['Cuaderno', 'Lápiz', 'Post-it'],
    ['Novela', 'Libro de Ciencia', 'Café'],
    ['Libro de Historia', 'Cuaderno'],
    ['Café', 'Post-it', 'Marcadores'],
    ['Novela', 'Lápiz'],
    ['Libro de Ciencia', 'Cuaderno', 'Post-it']
]

print("Dataset Original: \n")
for i, transaccion in enumerate(transacciones, 1):
    print(f"Transacción {i}: {transaccion}")

Dataset Original: 

Transacción 1: ['Novela', 'Café', 'Marcadores']
Transacción 2: ['Libro de Ciencia', 'Cuaderno', 'Lápiz']
Transacción 3: ['Novela', 'Café', 'Lápiz']
Transacción 4: ['Libro de Historia', 'Marcadores', 'Post-it']
Transacción 5: ['Cuaderno', 'Lápiz', 'Post-it']
Transacción 6: ['Novela', 'Libro de Ciencia', 'Café']
Transacción 7: ['Libro de Historia', 'Cuaderno']
Transacción 8: ['Café', 'Post-it', 'Marcadores']
Transacción 9: ['Novela', 'Lápiz']
Transacción 10: ['Libro de Ciencia', 'Cuaderno', 'Post-it']


### Preprocesamiento de Datos

In [ ]:
te = TransactionEncoder()

# Matriz booleana donde True indica la presencia del producto en la transacción
matriz = te.fit(transacciones).transform(transacciones)

df = pd.DataFrame(matriz, columns=te.columns_)

print("\nDataset Transormado (One-Hot Encoding): \n")
print(df)


Dataset Transormado (One-Hot Encoding): 



,Café,Cuaderno,Libro de Ciencia,Libro de Historia,Lápiz,Marcadores,Novela,Post-it
0,True,False,False,False,False,True,True,False
1,False,True,True,False,True,False,False,False
2,True,False,False,False,True,False,True,False
3,False,False,False,True,False,True,False,True
4,False,True,False,False,True,False,False,True
5,True,False,True,False,False,False,True,False
6,False,True,False,True,False,False,False,False
7,True,False,False,False,False,True,False,True
8,False,False,False,False,True,False,True,False
9,False,True,True,False,False,False,False,True


### Algoritmo Apriori. Items Frecuentes

In [ ]:
items_frecuentes = apriori(df, min_support=0.3, use_colnames=True)

print("\nItems Frecuentes (Soporte >= 30%): \n")
print(items_frecuentes.sort_values('support', ascending=False))

# Filtrar itemsets de tamaño 2
itemsets_tamano_2 = items_frecuentes[items_frecuentes['itemsets'].apply(lambda x: len(x) == 2)]

print("\n\nItems de Tamaño 2 (Soporte >= 30%): ")
print(itemsets_tamano_2.sort_values('support', ascending=False))


Items Frecuentes (Soporte >= 30%): 

   support            itemsets
0      0.4              (Café)
1      0.4          (Cuaderno)
3      0.4             (Lápiz)
5      0.4            (Novela)
6      0.4           (Post-it)
2      0.3  (Libro de Ciencia)
4      0.3        (Marcadores)
7      0.3      (Novela, Café)


Items de Tamaño 2 (Soporte >= 30%): 


,support,itemsets
7,0.3,"(Novela, Café)"


### Reglas de Asociación

In [ ]:
# Reglas con confianza mínima del 60%
reglas = association_rules(items_frecuentes, metric="confidence", min_threshold=0.6)

#Filtramos reglas con lift > 1.2
reglas = reglas[reglas['lift'] > 1.2]

print("\nReglas de Asociación (Confianza >= 60%):\n")
print(reglas[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values('lift', ascending=False))


Reglas de Asociación (Confianza >= 60%):



,antecedents,consequents,support,confidence,lift
0,(Novela),(Café),0.3,0.75,1.875
1,(Café),(Novela),0.3,0.75,1.875


### Preguntas Clave. Basado en la regla con mayor lift:

##### ¿Qué productos recomendarías empaquetar juntos en una promoción?

In [30]:
# Novela y Café, ya que comprar Novela aumenta la probabilidad de comprar 
# Café en un 87.5% respecto a lo normal.
# Esto quiere decir que si se compra Novela, hay un 75% probable de comprar Café o viceversa.

##### ¿Qué sección de la tienda podría reorganizarse para aumentar ventas cruzadas?

In [31]:
# La sección de novelas y la sección de café deberían ubicarse juntas o cerca para aumentar 
# las ventas cruzadas, ya que los clientes que compran novelas tienen alta probabilidad 
# de comprar café también.